In [ ]:
import numpy as np
import argparse
from overcooked_ai_py.agents.agent import AgentPair
from overcooked_ai_py.visualization.state_visualizer import StateVisualizer
from overcooked_ai_py.agents.benchmarking import AgentEvaluator

# from human_aware_rl.imitation.behavior_cloning_tf2 import (    _get_base_ae, BehaviorCloningPolicy)
from overcooked_ai_py.mdp.overcooked_env import OvercookedEnv, Overcooked
from overcooked_ai_py.mdp.overcooked_mdp import OvercookedGridworld
from tqdm.auto import trange
import warnings
from gen_env import GeneralizedOvercooked

from agent_policy import dumbAgent

# warnings.filterwarnings("ignore", category=UserWarning, module="pygame.pkgdata")

/media/leonardo/Data/Projects/AAS/project/AAS_project/.venv2/lib/python3.10/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


In [3]:
def normalize_reward(reward):
    warnings.warn("reward normalization not implemented")
    return reward


def clip_reward(reward):
    warnings.warni("reward clipping non implemented")
    return reward

In [ ]:
# ARGUMENTS:
MAX_LEN_EPISODE = 400  # max number of frames per episode
N_STEPS = 250  # ppo step = many rollouts + 1 backprop
BUFFER_LEN = MAX_LEN_EPISODE * 5  # 7 10
agent_name = "ppo"
save_path = f"weights"
lr = 1e-3  # 1e-4 2e-4 3e-4
layout_name = "cramped_room"
layout_name = "asymmetric_advantages"
layout_name = "coordination_ring"
discount_gamma = 0.99  # 0.995
advantage_lambda = 0.95
n_epochs = 7
epsilon_greedy = 0.02
val_epsilon_greedy = 1.0
epsilon_clip = 0.2

In [ ]:
# train agent
def train_agent():
    """"""
    env = GeneralizedOvercooked()
    my_agent = dumbAgent(buffer_len=BUFFER_LEN)

    score_history = np.zeros((MAX_LEN_EPISODE,))  # total reward of each step
    reward_history = np.zeros((MAX_LEN_EPISODE * N_STEPS,))  # single reward of each frame

    # step = play 1 episode (forward pass), log episode and train (backward pass)
    ## for each step
    state, obs = env.reset()
    # for step_n in trange(N_STEPS, colour="blue", desc="Training steps"):
    step_n = 0  # TODO iterazione con for perchè sappiamo quanti cicli dobbiamo fare
    while step_n < N_STEPS:
        print(f"training ppo-step {step_n}")
        state, obs = env.reset()
        print(f"finished game {42}")  # TODO
        episode_over, total_reward = False, 0

        # lenght of the episode is known/fixed
        ## for each frame in the episode
        for frame_n in trange(MAX_LEN_EPISODE):
            ## act
            obs1, obs2 = obs
            action1, prob1, value1 = my_agent.act(obs1, state)
            action2, prob2, value2 = my_agent.act(obs2, state)

            ## step
            next_state, next_obs, reward, episode_over, _info = env.step((action1, action2))

            ## buffer
            normalized_reward = normalize_reward(reward)
            clipped_reward = clip_reward(reward)
            # log reward for each action
            reward_history[(step_n * MAX_LEN_EPISODE) + frame_n] = reward  # TODO which reward: clipped, normalized?
            my_agent.store_step(
                state,
                obs1,
                obs2,
                action1,
                action2,
                prob1,
                prob2,
                value1,
                value2,
                normalized_reward,
                episode_over,
            )

            total_reward += reward  # TODO which reward? clipped? normalized?

            ## learn
            if my_agent.buffer_full():
                print(f"cumulative reward (score): ")
                my_agent.learn(n_epochs=7)
                step_n += 1
                if step_n % 3 == 0:  # TODO mettere a posto
                    my_agent.save()

            state, obs = next_state, next_obs

        # TODO add statistics: rewards non shaped (metrica)
        # log total scores for each episode
        score_history[step_n] = total_reward

    my_agent.save()
    env.close()
    # print/plot statistics